In [1]:
import os
import cv2 # Computer vision
import numpy as np # array

To be able to classify coins and banknotes we first need to feed training data to our model.
We first list all the paths to our images, then indecate the categories we will be using.

In [2]:
coin_datadir_train = '../coins-dataset/classified/train'
coin_datadir_test = '../coins-dataset/classified/test'
note_datadir_train = '../banknote-dataset/classified/train'
note_datadir_test = '../banknote-dataset/classified/test'

categories = ['1c', '2c', '5c', '10c', '20c', '50c', '1e', '2e', '5e', '10e', '20e', '50e']
coin_index = 8

There are two ways we could train our model, either by the dimensions of the coin or banknote (which can be very accurate) or by analyzing all the different color variations. To keep this simple there we will be analyzing the color variations.

To be able to analyze the colors of the photos, we made a loop that goes through all the photos and adds a list of the RGB values ​​of each photo into a list: training_data that contains a tuple of the RGB values and it's label indacating the category it belongs to

In [3]:
def resize_img(img):
    dim = (150, 150)
    new_img = cv2.resize(img, dim)
    return new_img

In [4]:
training_data = []


#add coins
for category in categories[:coin_index]:
    path = os.path.join(coin_datadir_train, category)
    label = 0
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        training_data.append([img_array, label])

#add notes
for category in categories[coin_index:]:
    path = os.path.join(note_datadir_train, category)
    label = 1
    for img in os.listdir(path):
        img_array = resize_img(cv2.imread(os.path.join(path, img)))
        training_data.append([img_array, label])


We do the same with seperate images to test the accuracy of our model

In [5]:
testing_data = []


#add coins
for category in categories[:coin_index]:
    path = os.path.join(coin_datadir_test, category)
    label = 0
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        testing_data.append([img_array, label])

#add notes
for category in categories[coin_index:]:
    path = os.path.join(note_datadir_test, category)
    label = 1
    for img in os.listdir(path):
        img_array = resize_img(cv2.imread(os.path.join(path, img)))
        testing_data.append([img_array, label])


We the seperate the labels from the RGB values

In [6]:
x_train = []
y_train = []

for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
    
x_train = np.array(x_train)

In [7]:
x_test = []
y_test = []

for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)
    
x_test = np.array(x_test)

The following function takes the RGB values of an image as input and then calculates the Euclidean distance between the image and all the other images in the training_data list. The k shortest distances are put in the votes list and then finally the function outputs the label that comes out the most often between the k iamges closest to the image whose value we want to know


In [8]:
from collections import Counter
def k_nearest_neighbors(predict, k):
    distances = []
    for image in training_data:
        distances.append([np.linalg.norm(image[0] - predict), image[1]]) # calcul de distance euclidienne
    distances.sort()
    votes = [i[1] for i in distances[:k]]
    votes = ''.join(str(e) for e in votes)
    votes = votes.replace(',', '')
    votes = votes.replace(' ', '')
    result = Counter(votes).most_common(1)[0][0]
    return result

We then test our model accuracy

In [22]:
from time import time
start = time()
correct = 0
total = 0
skipped = 0
for i in range(len(x_test)+1):
    try:
        prediction = k_nearest_neighbors(x_test[i], 5)
        if int(prediction) == y_test[i]:
            correct += 1
        total += 1
    except Exception as e:
        print('An exception occured')
        skipped += 1
accuracy = correct/total
end = time()
print(end-start)
print(accuracy)

An exception occured
527.3811609745026
0.931454196028187


Fianly we ask the user to upload an image and use our model to predict the value 

In [21]:
import tkinter.filedialog
from tkinter import *

root = Tk()
root.withdraw()
root.update()
filename = tkinter.filedialog.askopenfilename(title="Ouvrir fichier", filetypes=[('all files', '.*')]) # sélectionner la photo
src = cv2.imread(cv2.samples.findFile(filename), cv2.IMREAD_COLOR) # charger la photo
root.destroy()

img = resize_img(src)
pred = k_nearest_neighbors(img, 10)
if pred == '0':
    print('Coin')
else:
    print('Banknote')

Coin
